In [1]:
import os,sys
# from pathlib import Path
# Add the project root directory to Python path
# project_root = str(Path(__file__).parent.parent.parent)

sys.path.append("/home/doyooni303/experiments/sequential_recsys/ReLLMRec")
import json
from collections import defaultdict
from tqdm import tqdm


from src.utils import open_jsonl

In [2]:
dataset_path = "/data3/doyoon/sequential_recsys/datasets/"
dataset = "amazon"
category = "Books"
history = json.load(open(os.path.join(dataset_path, dataset, category,f"{category}_history.json"),'r'))

In [3]:
items_dict = json.load(open(os.path.join(dataset_path, dataset, category,f"{category}_itemidx2meta.json"),'r'))

In [6]:
len(items_dict), items_dict['0']

(4448181,
 {'main_category': 'Books',
  'title': 'Chaucer',
  'subtitle': 'Hardcover – Import, January 1, 2004',
  'author': {'avatar': 'https://m.media-amazon.com/images/I/21Je2zja9pL._SY600_.jpg',
   'name': 'Peter Ackroyd',
   'about': ['Peter Ackroyd, (born 5 October 1949) is an English biographer, novelist and critic with a particular interest in the history and culture of London. For his novels about English history and culture and his biographies of, among others, William Blake, Charles Dickens, T. S. Eliot and Sir Thomas More, he won the Somerset Maugham Award and two Whitbread Awards. He is noted for the volume of work he has produced, the range of styles therein, his skill at assuming different voices and the depth of his research.',
    'He was elected a fellow of the Royal Society of Literature in 1984 and appointed a Commander of the Order of the British Empire in 2003.',
    'Bio from Wikipedia, the free encyclopedia.']},
  'average_rating': 4.5,
  'rating_number': 29,
  

In [1]:
import random
from tqdm import tqdm
from typing import Tuple, List, Dict


import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import transformers
from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM
import numpy as np
import faiss

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(42)

In [13]:
help(torch.cosine_similarity)

Help on built-in function cosine_similarity in module torch:

cosine_similarity(...)
    cosine_similarity(x1, x2, dim=1, eps=1e-8) -> Tensor
    
    Returns cosine similarity between ``x1`` and ``x2``, computed along dim. ``x1`` and ``x2`` must be broadcastable
    to a common shape. ``dim`` refers to the dimension in this common shape. Dimension ``dim`` of the output is
    squeezed (see :func:`torch.squeeze`), resulting in the
    output tensor having 1 fewer dimension.
    
    .. math ::
        \text{similarity} = \dfrac{x_1 \cdot x_2}{\max(\Vert x_1 \Vert _2, \epsilon) \cdot \max(\Vert x_2 \Vert _2, \epsilon)}
    
    Supports :ref:`type promotion <type-promotion-doc>`.
    
    Args:
        x1 (Tensor): First input.
        x2 (Tensor): Second input.
        dim (int, optional): Dimension along which cosine similarity is computed. Default: 1
        eps (float, optional): Small value to avoid division by zero.
            Default: 1e-8
    
    Example::
    
        >>> inp

In [3]:
import torch
import time
from tqdm import tqdm

def get_top_k_similarities(query, candidates, k=10, batch_size=20000):
    """
    Calculate cosine similarity and return top-k matches for each query
    
    Args:
        query: tensor of shape (n_queries, embedding_dim)
        candidates: tensor of shape (n_candidates, embedding_dim)
        k: number of top matches to return
        batch_size: size of candidates batches to process at once
    
    Returns:
        top_scores: tensor of shape (n_queries, k) - top similarity scores
        top_indices: tensor of shape (n_queries, k) - indices of top matches
    """
    
    n_queries = query.shape[0]
    n_candidates = candidates.shape[0]
    
    # Normalize the vectors
    query_normalized = torch.nn.functional.normalize(query, p=2, dim=1)
    candidates_normalized = torch.nn.functional.normalize(candidates, p=2, dim=1)
    
    # Initialize tensors to store top-k scores and indices
    top_scores = torch.zeros((n_queries, k), device=query.device)
    top_indices = torch.zeros((n_queries, k), dtype=torch.long, device=query.device)
    
    # For each query, maintain a min-heap of top-k scores
    current_scores, current_indices = torch.topk(
        torch.zeros(n_queries, k, device=query.device),
        k=k, 
        dim=1, 
        largest=True
    )
    
    # Process in batches
    for i in tqdm(range(0, n_candidates, batch_size)):
        end_idx = min(i + batch_size, n_candidates)
        current_batch_size = end_idx - i
        
        # Calculate similarity for current batch
        batch_similarity = torch.matmul(
            query_normalized,
            candidates_normalized[i:end_idx].t()
        )
        
        # For each query, update top-k scores and indices
        if i == 0:
            # For first batch, simply get top-k
            top_scores, top_indices = torch.topk(batch_similarity, min(k, current_batch_size), dim=1)
            top_indices = top_indices + i  # Add offset
        else:
            # Concatenate previous top-k with current batch similarities
            all_scores = torch.cat([top_scores, batch_similarity], dim=1)
            all_indices = torch.cat([
                top_indices,
                torch.arange(i, end_idx, device=query.device).expand(n_queries, current_batch_size)
            ], dim=1)
            
            # Get top-k from all scores
            top_scores, indices = torch.topk(all_scores, k, dim=1)
            # Update top indices using the selected indices
            top_indices = torch.gather(all_indices, 1, indices)
    
    end = time.perf_counter()
    print(f"Processing took {end - start:.2f} seconds")
    
    return top_scores, top_indices

# Example usage:
device = torch.device(f'cuda')
query = torch.randn(16, 768).to(device)  # Your query tensor
candidates = torch.randn(4448181, 768).to(device)  # Your candidates tensor

# Get top 10 most similar candidates for each query
start = time.perf_counter()
top_scores, top_indices = get_top_k_similarities(query, candidates, k=10)

# Print results for first query
print("Top 10 similarity scores for first query:", top_scores[0])
print("Indices of top 10 matches for first query:", top_indices[0])

100%|██████████| 223/223 [00:00<00:00, 848.90it/s]

Processing took 0.49 seconds
Top 10 similarity scores for first query: tensor([0.1834, 0.1688, 0.1677, 0.1676, 0.1651, 0.1629, 0.1624, 0.1622, 0.1615,
        0.1613], device='cuda:0')
Indices of top 10 matches for first query: tensor([  98867,  989619,  517361, 3494341,  572961, 2909426, 3011890, 3786693,
        1731871, 4004155], device='cuda:0')


In [6]:
import torch
import torch.nn as nn
import torch.multiprocessing as mp
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from transformers import AutoModel, AutoTokenizer
from typing import List, Dict
import numpy as np
from tqdm import tqdm


class BookDataset(Dataset):
    def __init__(self, items: Dict, item_indices:List[str], meta_keys:List[str], tokenizer, max_length=4096):
        self.items = items
        self.item_indices = item_indices
        self.meta_keys = meta_keys
        self.max_length = max_length
        self.tokenizer = tokenizer
        

    def __len__(self):
        return len(self.items)

    def get_texts(self, item: dict, keys:List[str], category: str = "Books"):
        def _get_text_Books(item: dict, keys: list):
            text = ""
            for key in keys:
                try:
                    if isinstance(item[key],str):
                        text += f"{key}: {item[key]}</s>"
                    elif isinstance(item[key],list):
                        if len(item[key])>0:
                            add = ", ".join(item[key])
                            text += f"{key}: {add}</s>"
                    if key=='author':
                        text += f"author: {item['author']['name']}</s>"
                except:
                    text += f"{key}: None</s>"
            return text
        
        if category == "Books":
            text =  _get_text_Books(item, self.meta_keys)
        
        return text
            
    def __getitem__(self, idx:int):
        item = self.items[str(idx)]
        text = self.get_texts(item, self.meta_keys)
        
        encodings = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'item_idx': str(idx)
        }

class EmbeddingStorage(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        
    def forward(self, indices):
        return self.embedding(indices)
    
    def update_embeddings(self, indices: torch.Tensor, new_embeddings: torch.Tensor):
        """Update specific embeddings by indices"""
        with torch.no_grad():
            self.embedding.weight.data[indices] = new_embeddings


class EmbeddingGenerator:
    def __init__(self, 
                 model_name: str = 'allenai/longformer-base-4096',
                 batch_size: int = 8,
                 num_embeddings: int = 4448181,
                 embedding_dim: int = 768,
                 use_ddp: bool = False):  # Add this parameter
        
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim
        self.use_ddp = use_ddp
        
        # Initialize distributed environment if using DDP
        if self.use_ddp:
            if not dist.is_initialized():
                setup_distributed()
            self.rank = dist.get_rank()
            self.world_size = dist.get_world_size()
            self.device = torch.device(f'cuda:{self.rank}')
        else:
            self.rank = 0
            self.world_size = 1
            self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        
        # Set device
        if self.use_ddp:
            torch.cuda.set_device(self.device)
        
        self.model_name = model_name
        self.batch_size = batch_size
        
        # Initialize tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        base_model = AutoModel.from_pretrained(model_name)
        base_model.gradient_checkpointing_enable()
        
        # Move model to device
        self.model = base_model.to(self.device)
        
        # Wrap with DDP if using distributed training
        if self.use_ddp:
            self.model = DDP(self.model, 
                           device_ids=[self.rank],
                           output_device=self.rank)
        
        # Initialize embedding storage
        print("Initialize embedding storage...")
        self.embedding_storage = EmbeddingStorage(num_embeddings, embedding_dim)
        self.embedding_storage = self.embedding_storage.to(self.device)
        if self.use_ddp:
            self.embedding_storage = DDP(self.embedding_storage,
                                      device_ids=[self.rank],
                                      output_device=self.rank)

    def set_dataset_and_dataloader(self,
                                 items: dict,
                                 item_indices: List[str],
                                 keys: List[str]):
        # Create dataset
        self.dataset = BookDataset(items, item_indices, keys, self.tokenizer)
        
        # Create sampler based on DDP usage
        if self.use_ddp:
            self.sampler = DistributedSampler(self.dataset, 
                                            num_replicas=self.world_size,
                                            rank=self.rank)
        else:
            self.sampler = None
        
        self.dataloader = DataLoader(
            self.dataset,
            batch_size=self.batch_size,
            sampler=self.sampler,
            shuffle=(self.sampler is None),  # Only shuffle if not using DDP
            num_workers=4,
            pin_memory=True
        )
    def generate_and_store_embeddings(self, 
                                items: dict,
                                item_indices: List[str],
                                keys: List[str]):
        """
        Generate embeddings and store them in the embedding layer
        
        Args:
            items: List of item dictionaries containing metadata
            item_indices: List of indices corresponding to items
            keys: List of keys to extract from metadata
        """
        # Set dataset and dataloader if not already set
        if not hasattr(self, 'dataloader'):
            self.set_dataset_and_dataloader(items, item_indices, keys)
        
        # Generate embeddings in batches
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(self.dataloader, 
                            desc=f"Generating embeddings (GPU {self.device.index})"): 
                # Move batch to GPU
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                if isinstance(batch['item_idx'],List):
                    item_indices_batch = torch.tensor([int(idx) for idx in batch['item_idx']]).to(self.device, dtype=torch.int)
                                        
                # Get model outputs
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )
                
                # Get pooler output
                batch_embeddings = outputs.pooler_output
                
                # Update embedding storage
                # For single GPU, no need to access .module
                if hasattr(self.embedding_storage, 'module'):
                    self.embedding_storage.module.update_embeddings(
                        item_indices_batch,
                        batch_embeddings
                    )
                else:
                    self.embedding_storage.update_embeddings(
                        item_indices_batch,
                        batch_embeddings
                    )

        # Only call barrier if using DDP
        if dist.is_initialized():
            dist.barrier()

In [10]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"]= "0,2"  # Set the GPU 2 to use
os.environ['MASTER_ADDR'] = 'localhost'
os.environ['MASTER_PORT'] = '12355'

def setup_distributed(backend="nccl",
                      world_size: int = 2,
                      rank: int = 0):
    """Initialize distributed training environment"""
    print("Initialize distributed training environment")
    if not dist.is_initialized():
        dist.init_process_group(backend=backend,
                                world_size=world_size,
                                rank=rank,)
    
def cleanup_distributed():
    """Clean up distributed training environment"""
    if dist.is_initialized():
        dist.destroy_process_group()



# Define run function
def run_ddp(rank, world_size, model_name, items, item_indices, keys):
    # Note: rank is the first parameter spawned processes receive
    setup_distributed(world_size=world_size, rank=rank)
    generator = EmbeddingGenerator(model_name=model_name, use_ddp=True)
    generator.set_dataset_and_dataloader(items, item_indices, keys)
    generator.generate_and_store_embeddings(items, item_indices, keys)
    cleanup_distributed()

# if __name__ == "__main__":  # Add this for notebook

model_name = 'allenai/longformer-base-4096'
item_indices = list(items_dict.keys())
meta_keys = ['main_category',
'title',
'subtitle',
'author',
'average_rating',
'features',
'description',
'categories',]
num_embeddings = len(item_indices)
rank = 0
world_size = 2

mp.spawn(
    run_ddp,
    args=(world_size, model_name, items_dict, item_indices, meta_keys),
    nprocs=world_size
)

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/local/lib/python3.9/multiprocessing/spawn.py", line 116, in spawn_main
    exitcode = _main(fd, parent_sentinel)
  File "/usr/local/lib/python3.9/multiprocessing/spawn.py", line 126, in _main
    self = reduction.pickle.load(from_parent)
AttributeError: Can't get attribute 'run_ddp' on <module '__main__' (built-in)>


KeyboardInterrupt: 

In [ ]:

# For single GPU:
model_name = 'allenai/longformer-base-4096'
item_indices = list(items_dict.keys())
meta_keys = ['main_category',
 'title',
 'subtitle',
 'author',
 'average_rating',
 'features',
 'description',
 'categories',]
num_embeddings = len(item_indices)

use_ddp = True

generator = EmbeddingGenerator(model_name = model_name, use_ddp=use_ddp)
# generator.set_dataset_and_dataloader(items_dict, item_indices, meta_keys)
generator.generate_and_store_embeddings(items_dict, item_indices, meta_keys)

In [7]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
from transformers import LongformerModel, LongformerTokenizer
from typing import List, Dict
import numpy as np
from tqdm import tqdm


def setup_distributed(backend="nccl",
                      world_size: int = 2,
                      rank: int = 0):
    """Initialize distributed training environment"""
    print("Initialize distributed training environment")
    if not dist.is_initialized():
        dist.init_process_group(backend=backend,
                                world_size=world_size,
                                rank=rank,)
    
def cleanup_distributed():
    """Clean up distributed training environment"""
    if dist.is_initialized():
        dist.destroy_process_group()

class BookDataset(Dataset):
    def __init__(self, items: Dict, item_indices:List[str], meta_keys:List[str], tokenizer_name: str, max_length=4096):
        self.items = items
        self.item_indices = item_indices
        self.meta_keys = meta_keys
        self.max_length = max_length
        self.tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        

    def __len__(self):
        return len(self.items)

    def get_texts(self, item: dict, category: str = "Books"):
        def _get_text_Books(item: dict, keys: list):
            text = ""
            for key in keys:
                try:
                    if isinstance(item[key],str):
                        text += f"{key}: {item[key]}</s>"
                    elif isinstance(item[key],list):
                        if len(item[key])>0:
                            add = ", ".join(item[key])
                            text += f"{key}: {add}</s>"
                    if key=='author':
                        text += f"author: {item['author']['name']}</s>"
                except:
                    text += f"{key}: None</s>"
            return text
        
        if category == "Books":
            text =  _get_text_Books(item, self.meta_keys)
        
        return text
            
    def __getitem__(self, idx:int):
        item = self.items[str(idx)]
        text = self._get_text_Books(item, self.meta_keys)
        
        encodings = self.tokenizer(
            text,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encodings['input_ids'].squeeze(),
            'attention_mask': encodings['attention_mask'].squeeze(),
            'item_idx': str(idx)
        }

class EmbeddingStorage(nn.Module):
    def __init__(self, num_embeddings: int, embedding_dim: int):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        
    def forward(self, indices):
        return self.embedding(indices)
    
    def update_embeddings(self, indices: torch.Tensor, new_embeddings: torch.Tensor):
        """Update specific embeddings by indices"""
        with torch.no_grad():
            self.embedding.weight.data[indices] = new_embeddings

class DistributedEmbeddingGenerator:
    def __init__(self, 
                 model_name: str = 'allenai/longformer-base-4096',
                 batch_size: int = 8,
                 num_embeddings: int = 4448181,
                 embedding_dim: int = 768):
        
        self.num_embeddings = num_embeddings
        self.embedding_dim = embedding_dim

        # Initialize distributed environment if not already initialized
        if not dist.is_initialized():
            print("Initialize distributed environment if not already initialized")
            setup_distributed()
            
        self.rank = dist.get_rank()
        self.world_size = dist.get_world_size()
        
        # Set device
        self.device = torch.device(f'cuda:{self.rank}')
        torch.cuda.set_device(self.device)
        
        self.model_name = model_name
        self.batch_size = batch_size
        
        # Initialize tokenizer and model
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        base_model = AutoModel.from_pretrained(model_name)
        base_model.gradient_checkpointing_enable()
        
        # Move model to GPU
        self.model = base_model.to(self.device)
        
        # Wrap with DDP
        self.model = DDP(self.model, 
                        device_ids=[self.rank],
                        output_device=self.rank)
        
        # Initialize embedding storage
        print("Initialize embedding storage...")
        self.embedding_storage = EmbeddingStorage(num_embeddings, embedding_dim)
        self.embedding_storage = self.embedding_storage.to(self.device)
        self.embedding_storage = DDP(self.embedding_storage,
                                   device_ids=[self.rank],
                                   output_device=self.rank)

    def set_dataset_and_dataloader(self,
                                   items: dict,
                                   item_indices: List[str],
                                   keys: List[str]
                                   ):
        # Create dataset and distributed sampler
        self.dataset = BookDataset(items, item_indices, keys, self.tokenizer)
        self.sampler = DistributedSampler(dataset, 
                                   num_replicas=self.world_size,
                                   rank=self.rank)
        
        self.dataloader = DataLoader(
            dataset,
            batch_size=self.batch_size,
            sampler=self.sampler,
            num_workers=4,
            pin_memory=True
        )
        
    def generate_and_store_embeddings(self, 
                                    items: dict,
                                    item_indices: List[str],
                                    keys: List[str]):
        """
        Generate embeddings and store them in the embedding layer
        
        Args:
            items: List of item dictionaries containing metadata
            item_indices: List of indices corresponding to items
            keys: List of keys to extract from metadata
        """
        
        
        # Generate embeddings in batches
        self.model.eval()
        with torch.no_grad():
            for batch in tqdm(self.dataloader, 
                            desc=f"Generating embeddings (GPU {self.rank})",
                            disable=self.rank != 0):
                # Move batch to GPU
                input_ids = batch['input_ids'].to(self.device)
                attention_mask = batch['attention_mask'].to(self.device)
                item_indices_batch = batch['item_idx'].to(self.device, dtype=torch.int)
                
                # Get model outputs
                outputs = self.model(
                    input_ids=input_ids,
                    attention_mask=attention_mask,
                    output_hidden_states=True
                )
                
                # Get pooler output
                batch_embeddings = outputs.pooler_output
                
                # Update embedding storage
                self.embedding_storage.module.update_embeddings(
                    item_indices_batch,
                    batch_embeddings
                )
        
        # Synchronize processes
        dist.barrier()
    
    def save_embeddings(self, path: str):
        """Save the embedding layer state"""
        if self.rank == 0:
            torch.save(self.embedding_storage.module.state_dict(), path)
    
    def load_embeddings(self, path: str):
        """Load the embedding layer state"""
        state_dict = torch.load(path, map_location=self.device)
        self.embedding_storage.module.load_state_dict(state_dict)
        dist.barrier()

In [8]:
item_indices = list(items_dict.keys())
meta_keys = ['main_category',
 'title',
 'subtitle',
 'author',
 'average_rating',
 'features',
 'description',
 'categories',]
num_embeddings = len(item_indices)

# Initialize generator
generator = DistributedEmbeddingGenerator(
    model_name='allenai/longformer-base-4096',
    batch_size=8,
    num_embeddings=100000,
    embedding_dim=768
)

generator.set_dataset_and_dataloader(items_dict,
                                     item_indices,
                                     meta_keys)



Initialize distributed environment if not already initialized
Initialize distributed training environment


In [ ]:
# Generate and store embeddings
generator.generate_and_store_embeddings(items_dict, item_indices, meta_keys)

# Save embeddings (only on rank 0)
if dist.get_rank() == 0:
    generator.save_embeddings('embeddings.pt')
            
cleanup_distributed()

In [6]:
def get_text_Books(meta: dict, keys: list):
    text = ""
    for key in keys:
        try:
            if isinstance(meta[key],str):
                text += f"{key}: {meta[key]}</s>"
            elif isinstance(meta[key],list):
                if len(meta[key])>0:
                    add = ", ".join(meta[key])
                    text += f"{key}: {add}</s>"
            if key=='author':
                text += f"author: {meta['author']['name']}</s>"
        except:
            text += f"{key}: None</s>"
    return text

keys = ['main_category',
 'title',
 'subtitle',
 'author',
 'average_rating',
 'features',
 'description',
 'categories',]


tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
texts = [get_text_Books(item,keys) for item in items_dict.values()]

/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [7]:
lengths = np.array([len(text) for text in texts])
idx = np.argmax(lengths)

In [9]:
def set_seed(seed):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

class ItemEmbeddingSystem(nn.Module):
    def __init__(self, num_items: int, max_length: int, device: torch.device):
        super().__init__()
        set_seed(42)    
        self.num_items = num_items
        self.max_length = max_length
        self.device = device
        # Initialize Longformer for text processing
        self.tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
        self.longformer = AutoModel.from_pretrained('allenai/longformer-base-4096').to(self.device)
        self.embedding_dim = self.longformer.config.hidden_size
        
        # Create learnable embedding layer
        # self.item_embeddings = nn.Embedding(self.num_items, self.embedding_dim)
        self.item_embeddings = np.zeros((self.num_items,self.embedding_dim))
        # self.item_embeddings = np.empty((self.num_items,self.embedding_dim))

        # meta_keys
        self.meta_keys = {
            'Books':['title',
                    'subtitle',
                    'author',
                    'categories',
                    'average_rating',
                    'features',
                    # 'description',
                    ]
            }
            
    
    
        
            
    def create_item_text(self, item: Dict, main_category: str) -> str:
        def get_text_Books(meta: Dict, keys: list):
            text = ""
            for key in keys:
                try:
                    if isinstance(meta[key],str):
                        text += f"{key}: {meta[key]}</s>"
                    elif isinstance(meta[key],list):
                        if len(meta[key])>0:
                            add = ", ".join(meta[key])
                            text += f"{key}: {add}</s>"
                    if key=='author':
                        text += f"author: {meta['author']['name']}</s>"
                except:
                    text += f"{key}: None</s>"
            return text
        
        if main_category=="Books":
            keys = self.meta_keys['Books']
            text = get_text_Books(item, keys)
        
        return text

    def initialize_embeddings(self, items_dict: Dict[int, Dict], main_category, batch_size: int = 32):
        print("Initializing embeddings...")
        
        # Pre-process all texts at once
        all_texts = [self.create_item_text(item, main_category) for item in items_dict.values()]
        
        # Process in larger batches
        # batch_size = 128  # Increased batch size
        
        for batch_start in tqdm(range(0, len(all_texts), batch_size)):
            batch_texts = all_texts[batch_start:batch_start + batch_size]
            
            # Tokenize batch
            encoded = self.tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=self.max_length,
                return_tensors='pt'
            ).to(self.device)
            
            # Get embeddings
            batch_embedding = self.longformer(
                **encoded,
                return_dict=True
            ).pooler_output
            
            # Update embeddings
            indices = list(items_dict.keys())[batch_start:batch_start + batch_size]
            self.item_embeddings[batch_start:batch_start+batch_size]+= batch_embedding.detach().cpu().numpy()
            # self.item_embeddings[batch_start:batch_start+batch_size]= batch_embedding.detach().cpu().numpy()
            # for idx, item_idx in enumerate(indices):
                # self.item_embeddings.weight.data[int(item_idx)] = batch_embedding[idx]
                

    # def initialize_embeddings(self, items_dict: Dict[int, Dict], main_cateogry, batch_size: int = 32):
    #     """Initialize embeddings using Longformer in batches"""
    #     print("Initializing embeddings with Longformer...")
        
    #     with torch.no_grad():
    #         for batch_start in tqdm(range(0, len(items_dict), batch_size)):
    #             batch_items = {
    #                 idx: items_dict[idx] 
    #                 for idx in list(items_dict.keys())[batch_start:batch_start + batch_size]
    #             }
                
    #             # Process batch
    #             texts = [self.create_item_text(item, main_category) for item in batch_items.values()]
    #             encoded = self.tokenizer(
    #                 texts,
    #                 padding=True,
    #                 truncation=True,
    #                 max_length=self.max_length,
    #                 return_tensors='pt'
    #             )
                
    #             # Move to device
    #             input_ids = encoded['input_ids'].to(self.device)
    #             attention_mask = encoded['attention_mask'].to(self.device)
                
    #             # Get Longformer embeddings
    #             batch_embedding = self.longformer(
    #                 input_ids=input_ids,
    #                 attention_mask=attention_mask,
    #                 return_dict=True
    #             ).pooler_output
                
    #             # Update embedding layer
    #             for idx,item_idx in enumerate(batch_items.keys()):
    #                 self.item_embeddings.weight.data[int(item_idx)] = batch_embedding[idx]
        

    def forward(self, item_indices: torch.Tensor) -> torch.Tensor:
        """Get embeddings for given item indices"""
        return self.item_embeddings(item_indices)
    
    def save_embeddings(self, path: str):
        """Save embeddings to file"""
        torch.save(self.item_embeddings.state_dict(), path)
    
    def load_embeddings(self, path: str):
        """Load embeddings from file"""
        self.item_embeddings.load_state_dict(torch.load(path))


num_items = len(items_dict)
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
batch_size = 16
max_length = 1024
main_category="Books"
embedding_system = ItemEmbeddingSystem(num_items, max_length, device)
embedding_system.initialize_embeddings(items_dict, main_category, batch_size)


/usr/local/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Initializing embeddings...


  0%|          | 1/278012 [00:01<102:47:19,  1.33s/it]


OutOfMemoryError: CUDA out of memory. Tried to allocate 578.00 MiB. GPU 1 has a total capacity of 79.15 GiB of which 556.69 MiB is free. Process 2455427 has 78.57 GiB memory in use. Of the allocated memory 77.14 GiB is allocated by PyTorch, and 965.03 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)